In [1]:
from bs4 import BeautifulSoup
import os
import re

complete_text: list = []
root_directory = "."

for current_path, _, files in os.walk(root_directory):
    for filename in files:
        if filename.lower().endswith(".svg"):
            complet_path = os.path.join(current_path, filename)        
            content_svg = ""
            try:
                with open(complet_path, "r", encoding="utf-8") as f:
                    content_svg = f.read()
                soup = BeautifulSoup(content_svg, "xml")

                text_elements = soup.find_all("text")
                for text_element in text_elements:
                    text = text_element.get_text()
                    text = re.sub(r'^\s*$\n', "", text, flags=re.MULTILINE)
                    complete_text.append(text)

            except Exception as e:
                print(f"error when reading {complet_path}: {e}")

In [2]:
for text in complete_text:
    print(text)

Chapa Sigmóide
Apresentação
C ∫ Σ
ciencia_da_computacao@ufpa-belem

1º Tesoureiro
Luiz Henrique Mendes Costa

Semestre: 2º
Gosta de:
    - animes
 - manhwa
 - games
 - RPG
 - cibersegurança
 - curiosidades
      tecnológicas

henrique@ubuntu 

C ∫ Σ
1º Secretário
João Davi Costa de Souza
Semestre: 2º
Gosta de:
 - programação
 - software livre
 - música
 - sistemas
      operacionais
Environment:
 - Hyprland (WM)
 - Kitty (terminal)
 - VS Code (Editor)
jd@arch
C ∫ Σ
Tesoureiro Geral
Yuri Gabriel Cardoso Delgado
OBS: desenvolveu o estilo de apresentação
Semestre: 2º
Gosta de:
 - Linux
 - Ficção Científica
 - Low-Level
 - KPOP
 - TRON
Environment:
 - Hyprland (WM)
 - ST (terminal)
 - Neovim (Editor)
vanel@arch
C ∫ Σ

Secretário Geral

Semestre: 2º
Gosta de:
    - design
 - filmes
 - quadrinhos
Environment:
 - Sway (WM)
 - Foot (terminal)
 - Krita (para desenho)

kfk@garuda

Leonardo Brito da Conceição
OBS: desenvolveu a paleta e o símbolo da chapa

C ∫ Σ
Vice-Presidente
Lucas Santos Diniz

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=10,
    separators=["\n\n", "\n", " ", ""],
)

chunks = text_splitter.create_documents(complete_text)

In [4]:
print(f'number of chunks: {len(chunks)}')

for i,chunk in enumerate(chunks):
    print(f'Chunk {i+1}: len={len(chunk.page_content)}')
    print(chunk.page_content)

number of chunks: 197
Chunk 1: len=14
Chapa Sigmóide
Chunk 2: len=12
Apresentação
Chunk 3: len=5
C ∫ Σ
Chunk 4: len=32
ciencia_da_computacao@ufpa-belem
Chunk 5: len=13
1º Tesoureiro
Chunk 6: len=26
Luiz Henrique Mendes Costa
Chunk 7: len=114
Semestre: 2º
Gosta de:
    - animes
 - manhwa
 - games
 - RPG
 - cibersegurança
 - curiosidades
      tecnológicas
Chunk 8: len=15
henrique@ubuntu
Chunk 9: len=5
C ∫ Σ
Chunk 10: len=13
1º Secretário
Chunk 11: len=24
João Davi Costa de Souza
Chunk 12: len=166
Semestre: 2º
Gosta de:
 - programação
 - software livre
 - música
 - sistemas
      operacionais
Environment:
 - Hyprland (WM)
 - Kitty (terminal)
 - VS Code (Editor)
Chunk 13: len=7
jd@arch
Chunk 14: len=5
C ∫ Σ
Chunk 15: len=16
Tesoureiro Geral
Chunk 16: len=28
Yuri Gabriel Cardoso Delgado
Chunk 17: len=41
OBS: desenvolveu o estilo de apresentação
Chunk 18: len=147
Semestre: 2º
Gosta de:
 - Linux
 - Ficção Científica
 - Low-Level
 - KPOP
 - TRON
Environment:
 - Hyprland (WM)
 - ST (terminal)


In [5]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

model_name = "sentence-transformers/all-MiniLM-L6-v2"

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device":"cpu"}
)

vector_store = Chroma.from_documents(
    documents=chunks, 
    embedding=embeddings,
    persist_directory="./chroma_db"
)

retriever = vector_store.as_retriever()

In [6]:
query = "What kind of environment does Yuri use?"

relevant_docs = retriever.invoke(query)

print(f"\n '{query}'")

for i, doc in enumerate(relevant_docs):
    print(f"Document {i+1} (Context):")
    print(doc.page_content)


 'What kind of environment does Yuri use?'
Document 1 (Context):
Yuri Gabriel Cardoso Delgado
Document 2 (Context):
Semestre: 2º
Gosta de:
    - Tecnologia
 - Estudar
 - Séries
 - Filmes
 - Jogos
 - Desenhos
Environment:
 - Hyprland (WM)
 - Kitty (terminal)
 - VS Code (Editor)
Document 3 (Context):
O Tesoureiro Geral, Yuri Delgado, e o Secretário Geral, Leonardo Brito, apesar dos compromissos e de outros dificultantes, também demonstraram interesse em participar.
Document 4 (Context):
Secretário Geral


In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

from langchain_ollama import OllamaLLM
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv()
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

#llm = OllamaLLM(model="llama3")

prompt = ChatPromptTemplate.from_template("""
You are a helpful assistant. Use the following context to answer the question.
If you don't know the answer, say that you don't have enough information in the context.

Contexto:
{context}

Pergunta: {input}
""")

document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt,
)

rag_chain = create_retrieval_chain(retriever, document_chain)

query = "Tell me What kind of environment does Mateus use?."

response = rag_chain.invoke({"input": query})

print(f"\nresponse: {response['answer']}")


response: Mateus uses:
- **WM:** Hyprland
- **Terminal:** ST
- **Editor:** Neovim
